In [9]:
import pandas as pd
import tensorflow as tf
from glob import glob
from random import randint
DIR_WHERE_IMAGES_ARE="data/trimmed/*/*"
#In this case the classes are the 3rd level

def train_split_dataframes(dpath):
    df=pd.DataFrame.from_records([[g] for g in glob(DIR_WHERE_IMAGES_ARE)],columns=['file'])

    df['Class']=df.file.apply(lambda x: x.split('/')[2])
    df['Test']=False
    df['Train']=False

    df_cnt=df.groupby('Class').size().to_frame()
    df_cnt.columns=['Cnt']

    df=df.merge(df_cnt,on='Class')

    def set_train(max_occurences_in_class):
        #We want 66% to be training data
        r=randint(1,int(max_occurences_in_class))
        if r/max_occurences_in_class <= .61:
            return True
        else:
            return False
    df.Train = df.apply(lambda row: set_train(row['Cnt']),axis=1)
    df.Test = ~df.Train
    return df

df=train_split_dataframes(DIR_WHERE_IMAGES_ARE)
CLASSES=len(set(list(df.Class)))
print("Check Percentage {}".format(len(df[df.Test==True])/len(df[df.Test==False])))

df_test=df[df.Test==True]
df_train=df[df.Train==True]

# there should be non in both data sets
df_test.merge(df_train,on='file')

from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

SIZE_X = SIZE_Y = 70

datagen=ImageDataGenerator(rescale=1./255.)

# Train and Valid both come from TRAIN

train_generator=datagen.flow_from_dataframe(
dataframe=df_train,
x_col="file",
y_col="Class",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(SIZE_X,SIZE_Y))

valid_generator=datagen.flow_from_dataframe(
dataframe=df_train,
x_col="file",
y_col="Class",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(SIZE_X,SIZE_Y))

# Test comes from Test
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=df_test,
x_col="file",
y_col="Class",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(SIZE_X,SIZE_Y))

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(SIZE_X,SIZE_Y,3)))
model.add(Activation('relu'))
model.add(Conv2D(SIZE_X, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(CLASSES, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])


callbacks_wanted = [
  # Interrupt training if `val_loss` stops improving for over 2 epochs
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./logs', 
                                 histogram_freq=0,
                            write_graph=True, write_images=True)
]

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=callbacks_wanted,
                    epochs=10
)

model.save("birds.h5")

In [10]:
model.save?

Signature: model.save(filepath, overwrite=True, include_optimizer=True)
Docstring:
Saves the model to a single HDF5 file.

The savefile includes:
    - The model architecture, allowing to re-instantiate the model.
    - The model weights.
    - The state of the optimizer, allowing to resume training
        exactly where you left off.

This allows you to save the entirety of the state of a model
in a single file.

Saved models can be reinstantiated via `keras.models.load_model`.
The model returned by `load_model`
is a compiled model ready to be used (unless the saved model
was never compiled in the first place).

# Arguments
    filepath: String, path to the file to save the weights to.
    overwrite: Whether to silently overwrite any existing file at the
        target location, or provide the user with a manual prompt.
    include_optimizer: If True, save optimizer's state together.

# Example

```python
from keras.models import load_model

model.save('my_model.h5')  # creates a HDF5